In [10]:
print 'starting'

from robobrowser import RoboBrowser
from textblob import TextBlob
from pattern.es import tag
urls = 'http://www.elmostrador.cl/','http://www.elmostrador.cl/noticias/pais','http://www.elmostrador.cl/noticias/mundo','http://www.elmostrador.cl/noticias/opinion'
urls = urls[:1]
from pattern.vector import KNN, count
from collections import Counter, defaultdict
import indicoio
indicoio.config.api_key = '37958de57fa5f9d8d4bbec0c0902fd1e'

from string import uppercase

def is_person(txt):
    partes = txt.split(' ')
    if len(partes)<2:
        return False
    for parte in partes:
        try:
            if parte[0] not in uppercase:
                return False
        except:
            return False
    return True


browser = RoboBrowser(history=True, parser='html5lib')
palabras=Counter()
palabras_sentimiento = defaultdict(list)
articulos = []
for url in urls:
    print url
    browser.open(url)
    articulos = browser.select('h3 a') + browser.select('h2 a')
    for articulo in articulos:
        if '2016' not in articulo.attrs['href']:
            continue
        browser.follow_link(articulo)
        texto = browser.select('.cuerpo-noticia')
        if texto:
            texto = texto[0].text
        else:
            continue        
        titulo = browser.select('.datos-noticias h2')
        if titulo:
            titulo = titulo[0].text
        else:
            continue
        people = indicoio.people(texto)
        palabras = []
        for person in people:
            nombre = person['text']
            if is_person(nombre):
                #print nombre, person['confidence']
                palabras.append(nombre)
        if palabras:
            sentiment = indicoio.sentiment(texto, language='spanish')
            print u"%s" % titulo
            print palabras
            print sentiment
            print '----'
            for i in palabras:
                palabras_sentimiento[i].append(sentiment)

        


        


starting
http://www.elmostrador.cl/
Benito Mussolini 0.945443034172
Francesco Starace 0.658991277218
Matteo Renzi 0.600738525391
Achille Starace 0.0291506759822
El Mercurio 0.011916401796
La receta “fascista” de Starace para hacer negocios hace olas entre los que se opusieron a reorganización de Enersis
[u'Benito Mussolini', u'Francesco Starace', u'Matteo Renzi', u'Achille Starace', u'El Mercurio']
0.75137231785
----
Augusto Pinochet 0.908190131187
Mauricio Macri 0.424925088882
Chris Berry 0.0964323431253
Julio Ponce 0.0196659509093
Chile, el país más rico en litio del mundo puede quedar al margen del boom de Tesla Motors
[u'Augusto Pinochet', u'Mauricio Macri', u'Chris Berry', u'Julio Ponce']
0.776954145439
----
Joan Prats 0.0805134773254
Joan Prats-Catalá 0.0628228634596
Joan Prats-Catalá 0.0430718772113
Descentralización un tema serio y urgente
[u'Joan Prats', u'Joan Prats-Catal\xe1', u'Joan Prats-Catal\xe1']
0.889555954371
----
Michelle Bachelet 0.641868054867
Jorge Burgos 0.528574

In [11]:
puntajes = []
for palabra, sentimientos in palabras_sentimiento.items():
    if sentimientos:
        cantidad = len(sentimientos)
        if cantidad:
            promedio = sum(sentimientos)/cantidad
            puntajes.append([cantidad,promedio,palabra])

In [12]:
puntajes.sort()
puntajes.reverse()
puntajes[:100]

[[5, 0.7199278142296173, u'Michelle Bachelet'],
 [3, 0.9750473717118028, u'David Shrigley'],
 [3, 0.8562922178030125, u'Leonardo Farkas'],
 [3, 0.7941217524542203, u'Ricardo Lagos'],
 [3, 0.7185403674281208, u'Jorge Burgos'],
 [2, 0.9649253957939327, u'Joan Roca'],
 [2, 0.9323736170276081, u'David Rubilar'],
 [2, 0.8895559543714109, u'Joan Prats-Catal\xe1'],
 [2, 0.8811879500652017, u'Soledad Rodr\xedguez'],
 [2, 0.8794443395877242, u'Redes Neuronales'],
 [2, 0.8794443395877242, u'Francisco Stecher'],
 [2, 0.8468460282644065, u'TEATRO INFANTIL'],
 [2, 0.8181830878094514, u'Para Vivaldi'],
 [2, 0.7656088235470977, u'Jaime Orpis'],
 [2, 0.7637845089261217, u'Marcelo D\xedaz'],
 [2, 0.7175740546211898, u'Juan Pablo Larra\xedn'],
 [2, 0.6110767104724835, u'Patricio Aylwin'],
 [2, 0.5945967399734879, u'Gonzalo Navarrete'],
 [1, 0.9829971961371995, u'Cielos Chilenos'],
 [1, 0.9750473717118028, u'Stephen White'],
 [1, 0.9750473717118028, u'Stephen Friedman'],
 [1, 0.9750473717118028, u'Really

In [19]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.plotly as py
import numpy as np
py.sign_in('RodrigoCulagovski', 'hjhyib8mym')

x,y,text = zip(*puntajes)
trace = go.Scatter(
    x=x,
    y=y,
    text=text,
    textfont = {
        'size': 9
    },
    mode='markers+text',
    textposition = 'middle right',
    marker=dict(size=12,
                line=dict(width=1)
               ),
    name='Adjetivos',
    )
data = [trace]
layout = go.Layout(
    title='Personas',
    hovermode='closest',
    xaxis=dict(
        title='Frecuencia de mención',
        ticklen=5,
        zeroline=False,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Análisis de Sentimiento',
        ticklen=5,
        gridwidth=2,
    ),
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='palabras')